In [ ]:

url = "https://api.tokenterminal.com/v2/metrics"

headers = {
    "accept": "application/json",
    "authorization": "Bearer Your-API-Key"
}

response = requests.get(url, headers=headers)

print(response.text)

In [ ]:
# prompt: response.text to json

import json

data = json.loads(response.text)
# prompt: number of items in data

print(len(data['data']))

In [14]:
import requests
import json
import pandas as pd
import time

metrics = [
    'market_cap_fully_diluted',
    'market_cap_circulating',
    'token_trading_volume',
    'tokenholders',
    'tvl',
    'transaction_volume',
    'trading_volume',
    'transfer_volume',
    'fees',
    'fees_supply_side',
    'revenue',
    'expenses',
    'cost_of_revenue',
    'operating_expenses',
    'token_incentives',
    'gross_profit',
    'earnings',
    'voting_incentives',
    'treasury',
    'user_dau',
    'user_wau',
    'user_mau',
    'active_addresses_daily',
    'active_addresses_monthly',
    'active_developers',
    'code_commits',
    'trade_count',
    'transaction_count',
    'transactions_per_second',
    'transaction_fee_average',
    'gas_used',
    'trading_volume_avg_per_trade_count',
    'trading_volume_avg_per_user',
    'total_transaction_fees',
    'unique_transacting_wallets',
    'tokenholder_revenue'
]


Saved token_terminal_market_cap_fully_diluted.csv with 521570 records.
Saved token_terminal_market_cap_circulating.csv with 521237 records.
Saved token_terminal_token_trading_volume.csv with 571419 records.
Saved token_terminal_tokenholders.csv with 401159 records.
Saved token_terminal_tvl.csv with 146134 records.
Saved token_terminal_transaction_volume.csv with 7053 records.
Saved token_terminal_trading_volume.csv with 99388 records.
Saved token_terminal_transfer_volume.csv with 15431 records.
Saved token_terminal_fees.csv with 311070 records.
Saved token_terminal_fees_supply_side.csv with 269489 records.
Saved token_terminal_revenue.csv with 305547 records.
Saved token_terminal_expenses.csv with 249096 records.
Saved token_terminal_cost_of_revenue.csv with 14562 records.
Saved token_terminal_operating_expenses.csv with 5853 records.
Saved token_terminal_token_incentives.csv with 248242 records.
Saved token_terminal_gross_profit.csv with 14517 records.
Saved token_terminal_earnings.cs

In [30]:
def get_common_metrics(df, project_ids):
    """
    Returns a sorted list of metrics common across all specified project IDs.

    Parameters:
    - df (pd.DataFrame): DataFrame with columns 'metric_id' and 'project_id'
    - project_ids (list): List of project IDs to find common metrics for

    Returns:
    - List of common metric IDs
    """
    metric_sets = []

    for pid in project_ids:
        metrics = set(df[df['project_id'] == pid]['metric_id'])
        metric_sets.append(metrics)

    if not metric_sets:
        return []

    common_metrics = set.intersection(*metric_sets)
    return sorted(list(common_metrics))

# project_ids = ['bitcoin', 'ethereum', 'solana','compound']
# common_metrics = get_common_metrics(metric_project_mapping, project_ids)

# print(f"Common metrics for {project_ids} ({len(common_metrics)}):")
# print(common_metrics)


Common metrics for ['bitcoin', 'ethereum', 'solana', 'compound'] (17):
['active_addresses_daily', 'active_addresses_monthly', 'active_developers', 'code_commits', 'earnings', 'expenses', 'fees', 'fees_supply_side', 'market_cap_circulating', 'market_cap_fully_diluted', 'price', 'revenue', 'token_incentives', 'token_trading_volume', 'user_dau', 'user_mau', 'user_wau']


In [ ]:
import requests
import json
import pandas as pd
import time
import zipfile
import os

metrics = [

]

headers = {
    "accept": "application/json",
    "authorization": "Bearer Your-API-Key"
}

csv_files = []

for metric in metrics:
    url = f"https://api.tokenterminal.com/v2/metrics/{metric}?project_ids=all"

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        data = response.json()

        if 'data' in data and isinstance(data['data'], list) and data['data']:
            df = pd.DataFrame(data['data'])
            filename = f"token_terminal_{metric}.csv"
            df.to_csv(filename, index=False)
            csv_files.append(filename)
            print(f"Saved {filename} with {len(df)} records.")
        else:
            print(f"No data found for metric: {metric}")

    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch data for {metric}: {e}")

    time.sleep(15)

# ZIP all the CSVs
zip_filename = "token_terminal_data.zip"
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    for file in csv_files:
        zipf.write(file)
        print(f"Added {file} to {zip_filename}")

print(f"\nAll files zipped into {zip_filename}")


In [42]:
# prompt: method_of_aggregation=pd.read_csv('method_of_aggregation_of_metric.csv',delimeter="""\""")
# method\tmetric_id
# 0	as on date\tprice
# 1	as on date\tmarket_cap_fully_diluted
# 2	as on date\tmarket_cap_circulating
# 3	sum\ttoken_trading_volume
# 4	as on date\ttokenholders

import pandas as pd
method_of_aggregation = pd.read_csv('method_of_aggregation_of_metric.csv', delimiter="\t")
method_of_aggregation.head()


,method,metric_id
0,as on date,price
1,as on date,market_cap_fully_diluted
2,as on date,market_cap_circulating
3,sum,token_trading_volume
4,as on date,tokenholders


In [54]:
import os
import pandas as pd

# Create the 'preprocessed' directory if it doesn't exist
if not os.path.exists('preprocessed'):
    os.makedirs('preprocessed')
    print("Created directory: preprocessed")

# Iterate through the method_of_aggregation DataFrame
for index, row in method_of_aggregation.iterrows():
    metric_id = row['metric_id']
    aggregation_method = row['method']
    filename = f"token_terminal_{metric_id}.csv"
    filepath = os.path.join('.', filename)

    if os.path.exists(filepath):
        try:
            df_metric = pd.read_csv(filepath)

            # Ensure required columns exist
            if 'timestamp' in df_metric.columns and 'project_id' in df_metric.columns:
                # Handle timezone warning by removing timezone if any
                df_metric['timestamp'] = pd.to_datetime(df_metric['timestamp'], utc=True).dt.tz_localize(None)

                # Add year_month for grouping
                df_metric['year_month'] = df_metric['timestamp'].dt.to_period('M')

                # Sort for consistent first-row selection
                df_metric = df_metric.sort_values(by=['project_id', 'timestamp'])

                if aggregation_method == 'as on date':
                    # Select first row per project_id per month
                    grouped_df = (
                        df_metric
                        .groupby(['project_id', 'year_month'], as_index=False)
                        .first()
                    )

                elif aggregation_method == 'sum':
                    # Identify only numeric value columns to sum
                    value_cols = df_metric.select_dtypes(include='number').columns.difference(['timestamp']).tolist()
                    grouped_df = (
                        df_metric
                        .groupby(['project_id', 'year_month'], as_index=False)[value_cols]
                        .sum()
                    )

                else:
                    print(f"⚠️ Unsupported aggregation method '{aggregation_method}' for {metric_id}. Skipping.")
                    continue

                # Convert Period to string
                grouped_df['year_month'] = grouped_df['year_month'].astype(str)

                # Write to output
                output_filename = f"preprocessed_token_terminal_{metric_id}.csv"
                output_filepath = os.path.join('preprocessed', output_filename)
                grouped_df.to_csv(output_filepath, index=False)

                print(f"✅ Processed {aggregation_method} metric: {metric_id}")
            else:
                print(f"⚠️ 'timestamp' or 'project_id' missing in {filename}. Skipping.")
        except Exception as e:
            print(f"❌ Error processing {filename}: {e}")
    else:
        print(f"⚠️ File not found: {filename}. Skipping.")

✅ Processed as on date metric: price
✅ Processed as on date metric: market_cap_fully_diluted
✅ Processed as on date metric: market_cap_circulating
✅ Processed sum metric: token_trading_volume
✅ Processed as on date metric: tokenholders
✅ Processed as on date metric: tvl
✅ Processed sum metric: transaction_volume
✅ Processed sum metric: trading_volume
✅ Processed sum metric: transfer_volume
✅ Processed sum metric: fees
✅ Processed sum metric: fees_supply_side
✅ Processed sum metric: revenue
✅ Processed sum metric: expenses
✅ Processed sum metric: cost_of_revenue
✅ Processed sum metric: operating_expenses
✅ Processed sum metric: token_incentives
✅ Processed sum metric: gross_profit
✅ Processed sum metric: earnings
⚠️ File not found: token_terminal_voting_incentives.csv. Skipping.
✅ Processed as on date metric: treasury
✅ Processed as on date metric: user_dau
✅ Processed as on date metric: user_wau
✅ Processed as on date metric: user_mau
✅ Processed as on date metric: active_addresses_dai

In [57]:
# prompt: zip the preprocessed folder and download it
import zipfile
# Zip the 'preprocessed' folder
preprocessed_folder = 'preprocessed'
zip_filename = 'preprocessed.zip'

if os.path.exists(preprocessed_folder):
    with zipfile.ZipFile(zip_filename, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, _, files in os.walk(preprocessed_folder):
            for file in files:
                file_path = os.path.join(root, file)
                arcname = os.path.relpath(file_path, preprocessed_folder) # Keep directory structure within zip
                zipf.write(file_path, arcname)
    print(f"Successfully zipped '{preprocessed_folder}' into '{zip_filename}'.")
else:
    print(f"Error: Folder '{preprocessed_folder}' not found.")

# Download the zipped file (works in Colab/Jupyter)
from google.colab import files

if os.path.exists(zip_filename):
    files.download(zip_filename)
else:
    print(f"Error: Zip file '{zip_filename}' was not created.")


Successfully zipped 'preprocessed' into 'preprocessed.zip'.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>